**可识别且格式化输出** 
- 用户prompt
- AI收到promt，充当语义分析的角色
- 返回结构化的数据（收件人是谁，内容是啥）
- 开发者拿到数据执行操作

用户的输入是未知的，AI该怎么识别到用户此时是发邮件而不是做别的操作呢？如何更可靠地从模型中获取结构化数据？Function Calling就可以。

因为这两个模型微调了之后，可以检测何时应该调用函数（取决于输入）并使用符合函数签名的 JSON 进行响应。
- 模型层面，识别出何时需要调用函数来对输出格式化
- 函数方面，设定具体的格式化逻辑来让我们更好使用

回到发邮件的例子，现在可以看下，如果是用Function Calling，第一步的流程，还是先创建一个正常的conversation：

In [11]:
import json
from EmailSkill import send_email, send_email_action
from enum import Enum
from openai import OpenAI
client=OpenAI()

class SkillFunctions(Enum):
  SendEmail = 'send_email'

def chat_completions():
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      messages=[{"role": "user", "content": "send an email to my wife, telling her I'm not going to have dinner today."},],
      functions=[
        {
          "name": SkillFunctions.SendEmail.value,
          "description": "send email assistant",
          "parameters": {
            "type": "object",
            "properties": {
              "receiver": {
                "type": "string",
                "description": "收件人名字即可",
              },
              "content": {"type": "string", "description": "邮件的内容"},
            },
            "required": ["receiver", "content"],
          },
        }
      ],
      function_call="auto",
    )
    message = response["choices"][0].message
    print(message)    

In [12]:
chat_completions()

TypeError: 'ChatCompletion' object is not subscriptable